<b>The code below used STLM by using only Capacity field to predict the RUL(STLM using one variable with multisteps)</b> 
<p>We built the model only on Battery B0005</p>

In [1]:
import sys 
import numpy as np # linear algebra
from scipy.stats import randint
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv), data manipulation as in SQL
import matplotlib.pyplot as plt # this is used for the plot the graph 
import seaborn as sns # used for plot interactive graph. 
from sklearn.model_selection import train_test_split # to split the data into two parts
#from sklearn.cross_validation import KFold # use for cross validation
from sklearn.preprocessing import StandardScaler # for normalization
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline # pipeline making
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectFromModel
from sklearn import metrics # for the check the error and accuracy of the model
from sklearn.metrics import mean_squared_error,r2_score
from keras.callbacks import History 
## for Deep-learing:
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.utils import to_categorical
from keras.optimizers import SGD 
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
import itertools
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Dropout

Using TensorFlow backend.


In [2]:
df=pd.read_csv("B0005_discharge.csv")
#featurs=['Batt_name','cycle','amb_temp','voltage_battery','current_battery','temp_battery','current_load','voltage_load','time','Capacity','NewCap']
f1=['cycle','Capacity']
df=df[f1]
#dataset=df[(df.Batt_name =='B0005')| (df.Batt_name =='B0006') | (df.Batt_name =='B0007')|(df.Batt_name =='B0018')]
# Feature Selection
#dataset=df[(df['Batt_name']=='B0006_11')]
dataset=df#<70 need window to be 10
#dataset=dataset[f1]
train_num=91
data_train=dataset[(dataset['cycle']<train_num)]
data_set_train=data_train.iloc[:,1:2].values
data_test=dataset[(dataset['cycle']>=train_num)]
data_set_test=data_test.iloc[:,1:2].values
#Windoow Size after 90 could be 30 is a goood choise
from sklearn.preprocessing import MinMaxScaler
sc=MinMaxScaler(feature_range=(0,1))
data_set_train=sc.fit_transform(data_set_train)
data_set_test=sc.transform(data_set_test)

X_train=[]
y_train=[]
window_size=5 #int(train_num/2)
#cycle 82 ===> Wz:40==>pred: 78
#take the last 10t to predict 10t+1
#Cycle 110 ===> WZ:40 ===> batchz:32
for i in range(window_size,train_num-1):
    X_train.append(data_set_train[i-window_size:i,0])
    y_train.append(data_set_train[i,0])
X_train,y_train=np.array(X_train),np.array(y_train)
 
X_train=np.reshape(X_train,(X_train.shape[0],X_train.shape[1],1))


In [3]:
ln=len(data_train)
ln
print(len(X_train))

85


<h1> Applied STLM </h1>

In [4]:
regress=Sequential()
from keras import regularizers
regress.add(LSTM(units=200, return_sequences=True, input_shape=(X_train.shape[1],1)))
regress.add(Dropout(0.3))

regress.add(LSTM(units=200, return_sequences=True))
regress.add(Dropout(0.3))


regress.add(LSTM(units=200, return_sequences=True))
regress.add(Dropout(0.3))

regress.add(LSTM(units=200))
regress.add(Dropout(0.3))


regress.add(Dense(units=1))

regress.compile(optimizer='adam',loss='mean_squared_error',metrics=['mse'])

hist=regress.fit(X_train,y_train,epochs=100,batch_size=30)


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
85/85 [==============================] - 9s 110ms/step - loss: 0.4268 - mean_squared_error: 0.4268
Epoch 2/100
85/85 [==============================] - 0s 3ms/step - loss: 0.2779 - mean_squared_error: 0.2779
Epoch 3/100
85/85 [==============================] - 0s 3ms/step - loss: 0.0683 - mean_squared_error: 0.0683
Epoch 4/100
85/85 [==============================] - 0s 3ms/step - loss: 0.0538 - mean_squared_error: 0.0538
Epoch 5/100
85/85 [==============================] - 0s 3ms/step - loss: 0.0578 - mean_squared_error: 0.0578
Epoch 6/100
85/85 [==============================] - 0s 3ms/step - loss: 0.0626 - mean_squared_error: 0.0626
Epoch 7/100
85/85 [==============================] - 0s 3ms/step - loss: 0.0331 - mean_squared_error: 0.0331


85/85 [==============================] - 0s 2ms/step - loss: 0.0074 - mean_squared_error: 0.0074
Epoch 68/100
85/85 [==============================] - 0s 2ms/step - loss: 0.0056 - mean_squared_error: 0.0056
Epoch 69/100
85/85 [==============================] - 0s 2ms/step - loss: 0.0054 - mean_squared_error: 0.0054
Epoch 70/100
85/85 [==============================] - 0s 3ms/step - loss: 0.0050 - mean_squared_error: 0.0050
Epoch 71/100
85/85 [==============================] - 0s 3ms/step - loss: 0.0046 - mean_squared_error: 0.0046
Epoch 72/100
85/85 [==============================] - 0s 3ms/step - loss: 0.0055 - mean_squared_error: 0.0055
Epoch 73/100
85/85 [==============================] - 0s 2ms/step - loss: 0.0062 - mean_squared_error: 0.0062
Epoch 74/100
85/85 [==============================] - 0s 3ms/step - loss: 0.0057 - mean_squared_error: 0.0057
Epoch 75/100
85/85 [==============================] - 0s 2ms/step - loss: 0.0062 - mean_squared_error: 0.0062
Epoch 76/100
85/85 [===

In [9]:
#function to calculate the sum of the difference between two cycles.
def diff_avg(Data_set):
    data_sum=Data_set.iloc[:,1:2]
    avg=0
    init=data_sum['Capacity'][0]
    substr=0.0
    calcul=0.0
    for i in range(1,len(data_sum)):
        substr=init-data_sum['Capacity'][i]
        calcul=calcul+ substr
        init=data_sum['Capacity'][i]
    avg=calcul/len(data_sum)
    return (avg)
   


In [5]:
#sc=MinMaxScaler(feature_range=(0,1))
def Scale_X(dataset,lag):#return the 3 D array for training and 1 D to actual value.
   # from sklearn.preprocessing import MinMaxScaler
    data=dataset.iloc[:,1:2].values
    data=sc.fit_transform(data)
    X_train=[]
    y_train=[]
    window_size=lag
    train_num=len(data)
    for i in range(window_size,train_num-1):
        X_train.append(data[i-window_size:i,0])
        y_train.append(data[i,0])
    X_train,y_train=np.array(X_train),np.array(y_train)
    X_train=np.reshape(X_train,(X_train.shape[0],X_train.shape[1],1))
    return X_train, y_train

<h1> Test the Model that was built by STLM</h1>

In [6]:
len(data_test)
num_cycle_pred=5
cycles=len(data_train)+num_cycle_pred
print(cycles)
regress.loss
training_loss = hist.history['loss']
xx=0
for i in range(len(training_loss)):
    xx=xx+training_loss[i]
avg=xx/len(training_loss)
print(avg)
loss_train=training_loss[len(training_loss)-1]
ini_val=loss_train
print(loss_train)

95
0.016773545182967458
0.005426093552480726


In [7]:
#### predictions
#data_total=pd.concat((data_train['NewCap'],data_test['NewCap']),axis=0)
#inputs=data_total[len(data_total)-len(data_test)-10:].values
#inputs=inputs.reshape(-1,1)


In [10]:
#### predictions, firstly we predict the next 5 cycles, then we add these cycles to the training dataset
### After that, we retrain the model with t+1,t+2,t3,t4,t5. and we do again to predict the t6 -t10 and so on.
k=0
while k<1:
    
    data_total=data_train['Capacity']
    avg=diff_avg(data_train)
    test=[]
    l=[]
    test=data_total[len(data_total)-window_size:]
    #print(test)
    #Make prediction to the next 5 cycles
    for i in range(num_cycle_pred):
        inputs1=test[len(test)-window_size-i:]
        inputs=test[len(test)-window_size-i:].values
        inputs=inputs.reshape(-1,1)
        inputs=sc.transform(inputs)
        X_test=[]    
        X_test.append(inputs[i:window_size+i,0])
        X_test=np.array(X_test)
        X_test=np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))
        pred=regress.predict(X_test)
        pred=sc.inverse_transform(pred)
        pred=pred[:,0]
        
        if loss_train>= 0.0199:#this filter used only when the error 0.02 or above,(by assumption)
            xx=float(pred)-avg
        else:
            xx=float(pred)#-avg
        if i <= 5:
            just_test=inputs1[i:window_size+i]
            print(just_test)
            print(pred)
        ###########End of prediction###################
        ################################################
        #if loss_train>= 0.017:
         #   xx=float(pred)-avg#loss_train
        #elif ini_val>=1.07 and loss_train<0.017:
         #   xx=float(pred)+loss_train  
        #else:
         #   xx=float(pred)
        l.append(float(xx))
        y=test.index[len(test)-1]
        s1 = pd.Series(float(xx),index=[y+1])# convert the new prodiction to time series platform
        test=pd.concat([test,s1])
        j=len(data_train)+1
        s2=pd.Series([int(j),float(xx)],index=['cycle','Capacity'])# add new prediction to the Training dataset
        data_train=data_train.append(s2,ignore_index=True)
    X_train,y_train=Scale_X(data_train,window_size)# Scaling the training dataset with new cycles.
    hist=regress.fit(X_train,y_train,epochs=100,batch_size=30)# rebuild the model 
    training_loss = hist.history['loss']
    avg=diff_avg(data_train)# find avg(for thesum of the difference between two cycles)
    loss_train=training_loss[len(training_loss)-1]# get the RMSE
    k=k+1


85    1.527914
86    1.528525
87    1.522647
88    1.517486
89    1.605819
Name: Capacity, dtype: float64
[1.5144728]
86    1.528525
87    1.522647
88    1.517486
89    1.605819
90    1.514473
dtype: float64
[1.5170808]
87    1.522647
88    1.517486
89    1.605819
90    1.514473
91    1.517081
dtype: float64
[1.5206444]
88    1.517486
89    1.605819
90    1.514473
91    1.517081
92    1.520644
dtype: float64
[1.5274547]
89    1.605819
90    1.514473
91    1.517081
92    1.520644
93    1.527455
dtype: float64
[1.5371673]
Epoch 1/100
89/89 [==============================] - 1s 9ms/step - loss: 0.0058 - mean_squared_error: 0.0058
Epoch 2/100
89/89 [==============================] - 0s 3ms/step - loss: 0.0062 - mean_squared_error: 0.0062
Epoch 3/100
89/89 [==============================] - 0s 2ms/step - loss: 0.0051 - mean_squared_error: 0.0051
Epoch 4/100
89/89 [==============================] - 0s 3ms/step - loss: 0.0062 - mean_squared_error: 0.0062
Epoch 5/100
89/89 [===================

89/89 [==============================] - 0s 3ms/step - loss: 0.0041 - mean_squared_error: 0.0041
Epoch 71/100
89/89 [==============================] - 0s 3ms/step - loss: 0.0046 - mean_squared_error: 0.0046
Epoch 72/100
89/89 [==============================] - 0s 2ms/step - loss: 0.0046 - mean_squared_error: 0.0046
Epoch 73/100
89/89 [==============================] - 0s 2ms/step - loss: 0.0068 - mean_squared_error: 0.0068
Epoch 74/100
89/89 [==============================] - 0s 3ms/step - loss: 0.0058 - mean_squared_error: 0.0058
Epoch 75/100
89/89 [==============================] - 0s 3ms/step - loss: 0.0052 - mean_squared_error: 0.0052
Epoch 76/100
89/89 [==============================] - 0s 2ms/step - loss: 0.0056 - mean_squared_error: 0.0056
Epoch 77/100
89/89 [==============================] - 0s 3ms/step - loss: 0.0051 - mean_squared_error: 0.0051
Epoch 78/100
89/89 [==============================] - 0s 2ms/step - loss: 0.0056 - mean_squared_error: 0.0056
Epoch 79/100
89/89 [===